<a href="https://colab.research.google.com/github/Harshvijayvargiaya/Practical-Deep-Learning-for-Coders-2.0/blob/master/Text_classification_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization
import string
import re

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  22.0M      0  0:00:03  0:00:03 --:--:-- 22.0M


In [3]:
!rm -r aclImdb/test/unsup*
!rm -r aclImdb/test/urls*
!rm -r aclImdb/train/unsup*
!rm -r aclImdb/train/urls*
!rm -r aclImdb/test/label*
!rm -r aclImdb/train/label*
!rm -r aclImdb_v1.tar.gz

rm: cannot remove 'aclImdb/test/unsup*': No such file or directory


In [4]:
batch_size = 32 
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size = batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds=tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds= tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
    )
print(f"Number of batches in raw_train_ds : {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds : {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds : {raw_test_ds.cardinality()}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds : 625
Number of batches in raw_val_ds : 157
Number of batches in raw_test_ds : 782


In [5]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(5):
    print(text_batch.numpy()[i])
    print(label_batch.numpy()[i])

b'I\'ve seen tons of science fiction from the 70s; some horrendously bad, and others thought provoking and truly frightening. Soylent Green fits into the latter category. Yes, at times it\'s a little campy, and yes, the furniture is good for a giggle or two, but some of the film seems awfully prescient. Here we have a film, 9 years before Blade Runner, that dares to imagine the future as somthing dark, scary, and nihilistic. Both Charlton Heston and Edward G. Robinson fare far better in this than The Ten Commandments, and Robinson\'s assisted-suicide scene is creepily prescient of Kevorkian and his ilk. Some of the attitudes are dated (can you imagine a filmmaker getting away with the "women as furniture" concept in our oh-so-politically-correct-90s?), but it\'s rare to find a film from the Me Decade that actually can make you think. This is one I\'d love to see on the big screen, because even in a widescreen presentation, I don\'t think the overall scope of this film would receive its

In [6]:
from logging import StreamHandler
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
  return tf.strings.regex_replace(
      stripped_html, f"[{re.escape(string.punctuation)}]",""
  )


  # model cobstants
max_feature = 20000
embedding_dim= 128
sequence_length = 500

vectorize_layer = TextVectorization(
    standardize = custom_standardization,
    max_tokens= max_feature,
    output_mode="int",
    output_sequence_length = sequence_length,
)

text_ds = raw_train_ds.map(lambda x, y:x)
vectorize_layer.adapt(text_ds)

In [7]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label
# Vectorize the data.
train_ds=raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)
# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)


In [12]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,),dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_feature, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 128)         2560000   
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 conv1d (Conv1D)             (None, None, 128)         114816    
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         114816    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                             

In [15]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
625/625 [==============================] - 21s 13ms/step - loss: 0.5072 - accuracy: 0.7115 - val_loss: 0.3148 - val_accuracy: 0.8704
Epoch 2/3
625/625 [==============================] - 5s 8ms/step - loss: 0.2261 - accuracy: 0.9117 - val_loss: 0.3195 - val_accuracy: 0.8760
Epoch 3/3
625/625 [==============================] - 5s 8ms/step - loss: 0.1219 - accuracy: 0.9547 - val_loss: 0.4163 - val_accuracy: 0.8694


In [16]:
model.evaluate(test_ds)

782/782 [==============================] - 7s 9ms/step - loss: 0.4326 - accuracy: 0.8629


[0.4325501024723053, 0.8629199862480164]